In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Transform for CIFAR-10
transform = transforms.Compose([
    transforms.Resize((224, 224)), # Resizing for deeper architectures
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

100.0%
/Users/kuldeepsinh/Desktop/LAB_VI/DL/.venv/lib/python3.14/site-packages/torchvision/datasets/cifar.py:83: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  entry = pickle.load(f, encoding="latin1")


In [3]:
# Example Focal Loss implementation
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.ce = nn.CrossEntropyLoss()

    def forward(self, inputs, targets):
        logpt = self.ce(inputs, targets)
        pt = torch.exp(-logpt)
        loss = self.alpha * (1 - pt)**self.gamma * logpt
        return loss

In [5]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np

# Extract features and plot
def plot_tsne(features, labels, title):
    tsne = TSNE(n_components=2).fit_transform(features)
    plt.scatter(tsne[:, 0], tsne[:, 1], c=labels, cmap='tab10')
    plt.title(title)
    plt.colorbar()
    plt.show()

In [6]:
import torch.nn.functional as F

class ArcFaceLoss(nn.Module):
    def __init__(self, s=30.0, m=0.50):
        super(ArcFaceLoss, self).__init__()
        self.s = s
        self.m = m
        self.ce = nn.CrossEntropyLoss()

    def forward(self, logits, labels):
        # Normalizing logits to represent cosine similarity
        cosine = F.normalize(logits)
        theta = torch.acos(torch.clamp(cosine, -1.0 + 1e-7, 1.0 - 1e-7))
        
        # Add angular margin
        target_logit = torch.cos(theta + self.m)
        
        one_hot = torch.zeros_like(logits)
        one_hot.scatter_(1, labels.view(-1, 1).long(), 1.0)
        
        output = (one_hot * target_logit) + ((1.0 - one_hot) * cosine)
        output *= self.s
        return self.ce(output, labels)

In [7]:
def get_model(name, num_classes=10):
    if name == "VGGNet":
        return torchvision.models.vgg16(weights=None, num_classes=num_classes)
    elif name == "AlexNet":
        return torchvision.models.alexnet(weights=None, num_classes=num_classes)
    elif name == "ResNet":
        return torchvision.models.resnet50(weights=None, num_classes=num_classes)
    # Add other architectures (EfficientNet, MobileNet, etc.) as needed
    
def train_and_evaluate(model, train_loader, test_loader, optimizer, criterion, epochs, device):
    model.to(device)
    for epoch in range(epochs):
        model.train()
        train_correct = 0
        train_total = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

    # Final Test Accuracy
    model.eval()
    test_correct = 0
    test_total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()

    return (100 * train_correct / train_total), (100 * test_correct / test_total)

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
results = []

# Experiment 1: VGGNet + Adam + BCE (using CrossEntropy for multi-class)
vgg = get_model("VGGNet")
results.append(("VGGNet", "Adam", 10, "BCE", *train_and_evaluate(vgg, trainloader, testloader, optim.Adam(vgg.parameters()), nn.CrossEntropyLoss(), 10, device)))

# Experiment 2: AlexNet + SGD + Focal Loss
alex = get_model("AlexNet")
results.append(("AlexNet", "SGD", 20, "Focal Loss", *train_and_evaluate(alex, trainloader, testloader, optim.SGD(alex.parameters(), lr=0.01), FocalLoss(), 20, device)))

# Experiment 3: ResNet + Adam + ArcFace
res = get_model("ResNet")
results.append(("ResNet", "Adam", 15, "ArcFace", *train_and_evaluate(res, trainloader, testloader, optim.Adam(res.parameters()), ArcFaceLoss(), 15, device)))

# Display the Table
import pandas as pd
df = pd.DataFrame(results, columns=["Model", "Optimizer", "Epochs", "Loss Function", "Training Accuracy", "Testing Accuracy"])
print(df)

KeyboardInterrupt: 

In [ ]:
def extract_features(model, loader, device):
    model.eval()
    features = []
    labels_list = []
    # Hook or slice the model to get embeddings instead of class scores
    feature_extractor = torch.nn.Sequential(*(list(model.children())[:-1])) 
    
    with torch.no_grad():
        for i, (images, labels) in enumerate(loader):
            if i > 10: break # Small subset for visualization
            images = images.to(device)
            feat = feature_extractor(images).view(images.size(0), -1)
            features.append(feat.cpu().numpy())
            labels_list.append(labels.numpy())
            
    return np.concatenate(features), np.concatenate(labels_list)

# Run for ResNet + ArcFace
feats, lbls = extract_features(res, testloader, device)
plot_tsne(feats, lbls, "Feature Clusters (ResNet + ArcFace)")